In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader
import json
from datetime import datetime
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM
from apps.email_manager.models.email import EmailAttachment



In [2]:
convo_reader = ConversationReader(
    start_date=datetime(year=2024, month=2, day=10),
    end_date=datetime(year=2024, month=2, day=20),
)
convo_reader.extract_conversation()


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-10 00:00:00) while time zone support is active.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-20 00:00:00) while time zone support is active.
  warnings.warn(


In [3]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'INVOICE FOR INSURANCE COVER' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        insLLM = InsuranceERPLLM()
        insLLM.store_documents_in_vector_store(attachments)
        insLLM.search_documents_vector_store()
        insLLM.query_from_final_results(insLLM.document_query_results)
        # for attchmnt in attachments:
        #     print(attchmnt)
        # for a_msg in full_convo:
        #     print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `re

{
    "net_premium": {
        "Choice 1": "INVOICE NO. AMT -995-2024-02 \nINSURANCE POLICY / CERTIFICATE IN  1 ORIGINAL AND 1 COP Y \nFOR 1 10% INVOICE VALUE, BLANK ENDORSED SHOWING \nCLAIMS PAYABLE  AT DESTINATION IN CURRENCY OF TH E L/C, \nCOVERING ALL RISKS AS PER ICC (A) AND WAR RISKS FROM \nLOADING PORT TO DISCHARGE PORT , INDICATING THE NAME \nAND ADDRESS OF INSURANCE COMPANY\u2019S SURVAY IN CHINA .",
        "Choice 2": "INVOICE NO. AMT -995-2024-02 \nINSURANCE POLICY / CERTIFICATE IN  1 ORIGINAL AND 1 COP Y \nFOR 1 10% INVOICE VALUE, BLANK ENDORSED SHOWING \nCLAIMS PAYABLE  AT DESTINATION IN CURRENCY OF TH E L/C, \nCOVERING ALL RISKS AS PER ICC (A) AND WAR RISKS FROM \nLOADING PORT TO DISCHARGE PORT , INDICATING THE NAME \nAND ADDRESS OF INSURANCE COMPANY\u2019S SURVAY IN CHINA .",
        "Choice 3": "War Risk Rate Adjustment Clause,\nInstitute War Cancellation Clause (Cargo)\nInstitute Strikes Clauses (Cargo)\nInstitute Classification Clause\nSubject to Cargo Termination of

/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the en

=404 Not Found
-----
Answer: =2019-08-09
-----
=404 Not Found
-----
=404 Not Found
-----

------------------------------------



In [10]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'INVOICE FOR INSURANCE COVER' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        # insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        for attchmnt in attachments:
            print(attchmnt)
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break

/Users/mirbilal/Desktop/minsir/media/email_attachments/AMT-994-2024-02_dBnocPv.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/AMT-995-2024-02_BTEPAPf.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/INSURANCE_POLICY_AMT-995-2024-02_So5tDAZ.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/PR-0224-202504-M03-000152_btLlCci.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/PR-0224-202504-M03-000153_1dspkR8.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/AMT-994-2024-02_jA5t25U.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/AMT-995-2024-02_DeHnYuQ.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/INSURANCE_POLICY_AMT-995-2024-02_EvCFqfS.pdf
{
    "from": "waseem.shahzad@adamjeeinsurance.com",
    "to": [
        "shahzad@almuqeettextiles.com"
    ],
    "date": "February 19, 2024, 7:55 AM",
    "subject": "FW: INVOICE FOR INSURANCE COVER",
    "body": "Dear Mr. Shahzad\n\u00a0\nPlease check attachment Initial draft of

In [ ]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)